In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import requests
import responses
import urllib.request

In [3]:
DTGdata = pd.read_csv('./주행 데이터/편현구/편현구.csv')

In [4]:
vehicle_regulation = pd.read_csv('./vehicle_regulation.csv',index_col = 0)

In [5]:
hz = 5

In [6]:
DTGdata.drop(['time'],axis=1,inplace=True)

In [7]:
DTGdata.rename(columns={'Unnamed: 0':'time'},inplace=True)

In [20]:
def find_period(DTG_buffer):
    buffer = []
    if len(DTG_buffer) != 0:
        buffer.append(DTG_buffer[0])
        for i in range(1,len(DTG_buffer)):
            if ((DTG_buffer[i]-DTG_buffer[i-1]) > 1):
                buffer.append(DTG_buffer[i-1])
                buffer.append(DTG_buffer[i])
        buffer.append(DTG_buffer[-1])

    return buffer

In [21]:
def check_over_speed(DTGdata,vehicle):
    
    
    buffer_over_speed = DTGdata[(DTGdata['velocity'] > (vehicle_regulation[vehicle]['accel_lim'] + DTGdata['road_limit'])) & 
                               (DTGdata['road_limit']>0)].index
    buffer_long_accel = []
    
    for i in range(vehicle_regulation[vehicle]['long_accel_sec']*hz,len(DTGdata)):
        
        if(DTGdata[(DTGdata.index >= i - vehicle_regulation[vehicle]['long_accel_sec']*hz) & (DTGdata.index <i) & 
                   (DTGdata['velocity'] <= (vehicle_regulation[vehicle]['long_accel_lim'] + DTGdata['road_limit']))].empty):
            
            buffer_long_accel.append(i)
    
    buffer_over_speed_period = find_period(buffer_over_speed)
    buffer_long_accel_period = find_period(buffer_long_accel)
    
    if(len(buffer_over_speed)!=0):
        print(len(DTGdata)/hz,'초 중 ',
              len(buffer_over_speed)/hz,'초 동안 과속했습니다.')
    if(len(buffer_long_accel)!=0):
        print(len(DTGdata)/hz,'초 중 ',
              (len(buffer_long_accel)+vehicle_regulation[vehicle]['long_accel_sec'])/hz,'초 동안 장기과속했습니다.')

    return buffer_over_speed_period,buffer_long_accel_period

In [22]:
over_speed_period,long_accel_period = check_over_speed(DTGdata,'truck')

539.6 초 중  58.4 초 동안 과속했습니다.


In [23]:
np.array(over_speed_period)

array([ 984, 1023, 1628, 1631, 1795, 1833, 1958, 2015, 2169, 2295, 2314,
       2314, 2418, 2440], dtype=int64)

In [24]:
long_accel_period

[]

In [25]:
def check_rapid_accel(DTGdata,vehicle):
    
    buffer_rapid_accel = []
    buffer_rapid_start = []
    
    for i in range(1*hz,len(DTGdata)):
        
        is_rapid_accel = (((DTGdata['velocity'][i]-DTGdata['velocity'][i-1*hz]) >= vehicle_regulation[vehicle]['rapid_accel_acceleration']) &
                         (DTGdata['velocity'][i] >= vehicle_regulation[vehicle]['rapid_accel_threshold']))
        
        is_rapid_start = (((DTGdata['velocity'][i]-DTGdata['velocity'][i-1*hz]) >= vehicle_regulation[vehicle]['rapid_start_acceleration']) &
                         (DTGdata['velocity'][i] <= vehicle_regulation[vehicle]['rapid_start_threshold']))
        
        if is_rapid_accel==1:
            buffer_rapid_accel.append(i)
         
        if is_rapid_start==1:
            buffer_rapid_start.append(i)
    
    buffer_rapid_accel_period = find_period(buffer_rapid_accel)
    buffer_rapid_start_period = find_period(buffer_rapid_start)
    
    
    if(len(buffer_rapid_accel)!=0):
        print(len(DTGdata)/hz,'초 중 ',len(buffer_rapid_accel)/hz,'초 동안 급가속했습니다.')
    if(len(buffer_rapid_start)!=0):
        print(len(DTGdata)/hz,'초 중 ',len(buffer_rapid_start)/hz,'초 동안 급출발했습니다.')

    return buffer_rapid_accel_period,buffer_rapid_start_period

In [26]:
check_rapid_accel(DTGdata,'truck')

539.6 초 중  27.8 초 동안 급가속했습니다.
539.6 초 중  1.8 초 동안 급출발했습니다.


([72,
  77,
  85,
  88,
  94,
  95,
  104,
  107,
  150,
  160,
  165,
  168,
  171,
  174,
  295,
  309,
  313,
  315,
  391,
  395,
  437,
  443,
  445,
  451,
  455,
  457,
  466,
  468,
  508,
  515,
  520,
  528,
  2308,
  2308,
  2328,
  2335,
  2339,
  2341,
  2351,
  2352,
  2459,
  2463,
  2466,
  2472,
  2477,
  2480,
  2609,
  2614,
  2624,
  2627,
  2632,
  2634,
  2677,
  2677],
 [61, 61, 63, 65, 67, 67, 251, 254])

In [27]:
def check_rapid_brake(DTGdata,vehicle):
    
    buffer_rapid_decel = []
    buffer_rapid_stop = []
    
    for i in range(1,len(DTGdata)):
        
        is_rapid_decel = (((DTGdata['velocity'][i-1]-DTGdata['velocity'][i])*hz >= vehicle_regulation[vehicle]['rapid_decel_deceleration']) &
                          (DTGdata['velocity'][i] >= vehicle_regulation[vehicle]['rapid_decel_threshold']))
        is_rapid_stop = (((DTGdata['velocity'][i-1]-DTGdata['velocity'][i])*hz >= vehicle_regulation[vehicle]['rapid_stop_deceleration']) &
                         (DTGdata['velocity'][i] <= vehicle_regulation[vehicle]['rapid_stop_threshold']))
        
        if is_rapid_decel :
            buffer_rapid_decel.append(i)
            
        if is_rapid_stop :
            buffer_rapid_stop.append(i)
            
    buffer_rapid_decel_period = find_period(buffer_rapid_decel)
    buffer_rapid_stop_period = find_period(buffer_rapid_stop)
            
            
    if(len(buffer_rapid_decel)!=0):
        print(len(DTGdata)/hz,'초 중 ',len(buffer_rapid_decel)/hz,'초 동안 급감속했습니다.')
    if(len(buffer_rapid_stop)!=0):
        print(len(DTGdata)/hz,'초 중 ',len(buffer_rapid_stop)/hz,'초 동안 급정지했습니다.')

    return buffer_rapid_decel_period,buffer_rapid_stop_period

In [28]:
np.array(check_rapid_brake(DTGdata,'truck'))

539.6 초 중  28.8 초 동안 급감속했습니다.
539.6 초 중  5.8 초 동안 급정지했습니다.


array([list([126, 133, 140, 140, 226, 230, 365, 372, 377, 378, 497, 501, 683, 685, 688, 696, 1021, 1028, 1094, 1095, 1130, 1135, 1217, 1223, 1273, 1278, 1411, 1419, 1636, 1641, 1831, 1839, 2294, 2300, 2310, 2322, 2439, 2447, 2449, 2451, 2542, 2547, 2666, 2671, 2679, 2682, 2690, 2691]),
       list([17, 17, 45, 48, 52, 52, 58, 60, 62, 62, 134, 134, 141, 143, 231, 232, 244, 245, 379, 380, 398, 400, 2452, 2453, 2548, 2549, 2583, 2583, 2692, 2692])],
      dtype=object)

In [29]:
def check_rapid_lane_change(DTGdata,vehicle):
    
    buffer_rapid_lanechange = []
    buffer_rapid_overtake = []
    
    for i in range(vehicle_regulation[vehicle]['rapid_lanechange_time'],len(DTGdata)):
        
        is_rapid_lanechange = ((DTGdata['velocity'][i] >= vehicle_regulation[vehicle]['rapid_lanechange_threshold']) & 
                               (abs(DTGdata['heading'][i] - DTGdata['heading'][i-1]) >= vehicle_regulation[vehicle]['rapid_lanechange_angle']) & 
                               (abs(DTGdata['heading'][i] - DTGdata['heading'][i-5])/vehicle_regulation[vehicle]['rapid_lanechange_time'] <= vehicle_regulation[vehicle]['rapid_lanechange_cangle']) &
                               (abs(DTGdata['velocity'][i] - DTGdata['velocity'][i-1]) <= vehicle_regulation[vehicle]['rapid_lanechange_defference']))
                               
        is_rapid_overtake = ((DTGdata['velocity'][i] >= vehicle_regulation[vehicle]['rapid_overtake_threshold']) & 
                               (abs(DTGdata['heading'][i] - DTGdata['heading'][i-1]) >= vehicle_regulation[vehicle]['rapid_overtake_angle']) & 
                               (abs(DTGdata['heading'][i] - DTGdata['heading'][i-5])/vehicle_regulation[vehicle]['rapid_lanechange_time'] <= vehicle_regulation[vehicle]['rapid_overtake_cangle']) &
                               (abs(DTGdata['velocity'][i] - DTGdata['velocity'][i-1])<= vehicle_regulation[vehicle]['rapid_overtake_acceleration']))
        
        if is_rapid_lanechange:
            buffer_rapid_lanechange.append(i)
            
        if is_rapid_overtake:
            buffer_rapid_overtake.append(i)
            
    if(len(buffer_rapid_lanechange)!=0):
        print(len(DTGdata),'초 중 ',len(buffer_rapid_lanechange),'초 동안 급차선변경했습니다.')
    if(len(buffer_rapid_overtake)!=0):
        print(len(DTGdata),'초 중 ',len(buffer_rapid_overtake),'초 동안 급앞지르기했습니다.')

    return len(buffer_rapid_lanechange), len(buffer_rapid_overtake)

In [30]:
check_rapid_lane_change(DTGdata,'taxi')

(0, 0)

In [31]:
def check_rapid_turn(DTGdata,vehicle):
    
    buffer_rapid_turn = []
    buffer_rapid_uturn = []
    turn_time = vehicle_regulation[vehicle]['rapid_turn_sec']
    uturn_time = vehicle_regulation[vehicle]['rapid_uturn_sec']
    
    for i in range(vehicle_regulation[vehicle]['rapid_uturn_sec'],len(DTGdata)):
        
        is_rapid_turn = ((DTGdata['velocity'][i] >= vehicle_regulation[vehicle]['rapid_turn_threshold']) &
                         (abs(DTGdata['heading'][i] - DTGdata['heading'][i-turn_time]) >= vehicle_regulation[vehicle]['rapid_turn_angle_start'])&
                         (abs(DTGdata['heading'][i] - DTGdata['heading'][i-turn_time]) <= vehicle_regulation[vehicle]['rapid_turn_angle_end'])) 
        is_rapid_uturn = ((DTGdata['velocity'][i] >= vehicle_regulation[vehicle]['rapid_uturn_threshold']) &
                         (abs(DTGdata['heading'][i] - DTGdata['heading'][i-uturn_time]) >= vehicle_regulation[vehicle]['rapid_uturn_angle_start'])&
                         (abs(DTGdata['heading'][i] - DTGdata['heading'][i-uturn_time]) <= vehicle_regulation[vehicle]['rapid_uturn_angle_end'])) 
        
        if is_rapid_turn:
            buffer_rapid_turn.append(i)
            
        if is_rapid_uturn:
            buffer_rapid_uturn.append(i)
            
    if(len(buffer_rapid_turn)!=0):
        print(lend(DTGdata),'초 중 ',len(buffer_rapid_turn),'초 동안 급 좌/우회전했습니다.')
    if(len(buffer_rapid_uturn)!=0):
        print(lend(DTGdata),'초 중 ',len(buffer_rapid_uturn),'초 동안 급 U턴했습니다.')

In [32]:
check_rapid_turn(DTGdata,'taxi')

In [33]:
def check_long_term(DTGdata,vehicle):
    
    driving_time = len(DTGdata)
    rest_time = len(DTGdata[(DTGdata['rpm']==0) & (DTGdata['velocity']==0)])
    
    if(driving_time >= vehicle_regulation[vehicle]['long_term_time']) & (rest_time <= vehicle_regulation[vehicle]['long_term_rest']):
        print(driving_time,'초 중 ',rest_time,'초 동안 휴식했습니다.')
        return driving_time,rest_time
    
    else: return 0

In [34]:
check_long_term(DTGdata,'taxi')

0

In [35]:
def check_regulation(DTGdata,vehicle):

    check_over_speed(DTGdata,vehicle)
    check_rapid_accel(DTGdata,vehicle)
    check_rapid_brake(DTGdata,vehicle)
    check_rapid_lane_change(DTGdata,vehicle)
    check_rapid_turn(DTGdata,vehicle)
    check_long_term(DTGdata,vehicle)

In [36]:
check_regulation(DTGdata,'taxi')

539.6 초 중  58.4 초 동안 과속했습니다.
539.6 초 중  3.6 초 동안 급가속했습니다.
539.6 초 중  0.4 초 동안 급출발했습니다.
539.6 초 중  27.0 초 동안 급감속했습니다.
539.6 초 중  7.6 초 동안 급정지했습니다.
